# 模型构造
## 继承Module类来构造模型
给予Module类的模型构造方法：它让模型构造更加灵活

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层， 这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Block的构造函数来进行必要的初始化，这样在构造实例时还可以指定
        # 其他函数参数，如模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256)   # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)   # 输出层
        
        
    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [2]:
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0241,  0.2113,  0.0124, -0.1108, -0.1169,  0.0222, -0.2923, -0.3411,
          0.0743,  0.0111],
        [-0.0083,  0.1894,  0.1263, -0.1096, -0.0178,  0.0590, -0.1666, -0.3051,
          0.0527,  0.1666]], grad_fn=<AddmmBackward>)

Module类是一个可供自由组建的部件，它的子类既可以是一个层，又可以是一个模型，或者是模型的一个部分。

## Module的子类
### Sequential类


In [3]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        # 如果传入的是一个OrderedDict
        if len(args) == 1 and isinstance(args[0], OrderedDict):
            for key, module in args[0].items():
                self.add_module(key, module)
        else:
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
                
    def forward(self, input):
        # self.add_module返回一个OrderedDict，保证会按照成员添加时的顺序遍历
        for module in self._modules.values():
            input = module(input)
        return input

In [4]:
net = MySequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 10))
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.0317,  0.0402, -0.2891, -0.0342, -0.1466,  0.0490,  0.1630, -0.1218,
          0.2359,  0.2546],
        [-0.0911,  0.0758, -0.3241,  0.1013, -0.0801,  0.0434,  0.2156,  0.0767,
          0.2005,  0.1867]], grad_fn=<AddmmBackward>)

ModuleList接收一个子模块的列表作为输入，也可以类似于List那样进行append和extend操作

In [5]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


MoudleDict接收一个子模块的字典作为输入，也可以类似字典进行添加访问操作

In [6]:
net = nn.ModuleDict({
    'linear' : nn.Linear(784, 256),
    'act' : nn.ReLU(),
})
net['output'] = nn.Linear(256, 10)  # 添加
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


## 构造复杂的模型
构造一个稍微复杂点的⽹网络 FancyMLP 。在这个网络中，我们通过 get_constant 函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了了使用创建的常数参数外，还使用Tensor 的函数和Python的控制流，并多次调用相同的层。

In [7]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        # 不可训练参数（常数参数）
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        # 复用全连接层，等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
            
        return x.sum()

In [8]:
# 测试该模型的前向计算
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(16.4237, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        # FancyMLP和Sequential类都是Module的子类，可以嵌套调用它们
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())
        
    def forward(self, x):
        return self.net(x)
    
net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-4.2609, grad_fn=<SumBackward0>)

# 模型参数的访问、初始化和共享
## 模型参数的访问

In [10]:
import torch
from torch import nn
from torch.nn import init

# pytorch已进行默认初始化
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [11]:
# 访问模型参数
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [13]:
# 访问net中单层的参数
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [14]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        # Parameter会被自动添加到模型的参数列表中
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
        
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


## 初始化模型参数
PyTorch的 init 模块里提供了多种预设的初始化方法。

In [16]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0031,  0.0059, -0.0086, -0.0063],
        [ 0.0122,  0.0180, -0.0199, -0.0051],
        [-0.0020, -0.0074, -0.0098, -0.0120]])
2.weight tensor([[ 0.0007, -0.0014,  0.0050]])


In [17]:
# 使用常数来初始化权重参数
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 自定义初始化方法


In [18]:
# pytorch的初始化方法
def normal_(tensor, mean=0, std=1):
    # 不记录梯度
    with torch.no_grad():
        return tensor.normal_(mean, std)

自定义初始化方法：我们令权重有一半概率初始化为0，有另一半概率初始化为\[-10, -5\] 和\[5, 10\] 两个区间里均匀分布的随机数

In [28]:
def init_weight_(tensor):
    with torch.no_grad():
        # 数值统一约束
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()
        
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000,  0.0000,  5.9449, -0.0000],
        [ 0.0000, -6.2289, -6.2491,  0.0000]])
2.weight tensor([[-0.0000, 9.4037, 5.9882]])


In [29]:
# 通过改变这些参数的data来改写模型参数值同时不会影响梯度
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 共享模型参数
共享模型参数的情况：Module类的forward函数多次调用同一个层；如果我们传入Sequential的模块是同一个Module实例的话参数也是共享的

In [30]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


在内存中，这两个线性层其实是一个对象

In [31]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:

In [32]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


# 自定义层
## 不含模型参数的自定义层
下⾯面的 CenteredLayer 类通过继承 Module 类自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了 forward 函数里。这个层里不含模型参数。

In [33]:
class CenteredLayer(nn.Module):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
        
    def forward(self, x):
        return x - x.mean()

In [34]:
# 实例化这个层，然后做前向计算
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

用它来构造更复杂的模型

In [35]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [36]:
y = net(torch.rand(4, 8))
y.mean().item()

-4.6566128730773926e-09

## 含模型参数的自定义层
Parameter 类其实是 Tensor 的子类，如果一个 Tensor 是 Parameter ，那么它会自动被添加到模型的参数列表里。所以在自定义含模型参数的层时，我们应该将参数定义成 Parameter。  
直接定义成 Parameter 类外，还可以使用 ParameterList 和 ParameterDict 分别定义参数的列表和字典。

ParameterList 接收一个 Parameter 实例的列表作为输⼊入然后得到一个参数列列表，使用的时候可以用索引来访问某个参数，另外也可以使用 append 和 extend 在列表后面新增参数。

In [37]:
class MyDense(nn.Module):
    def __init__(self):
        super(MyDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4))
                                        for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))
        
        
    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
    
net = MyDense()
print(net)

MyDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


而 ParameterDict 接收一个 Parameter 实例的字典作为输入然后得到一个参数字典，然后可以按照字典的规则使用了。

In [41]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
            'linear1' : nn.Parameter(torch.randn(4, 4)),
            'linear2' : nn.Parameter(torch.randn(4, 1))
        })
        # 新增
        self.params.update({'linear3' : nn.Parameter(torch.randn(4, 2))})
        
    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])
    
net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [42]:
# 根据出入的键值来进行不同的前向传播
x = torch.ones(1, 4)
print(net(x, 'linear1'))
print(net(x, 'linear2'))
print(net(x, 'linear3'))

tensor([[ 0.0543, -1.5141, -0.1236, -0.0570]], grad_fn=<MmBackward>)
tensor([[0.9412]], grad_fn=<MmBackward>)
tensor([[0.8352, 0.7251]], grad_fn=<MmBackward>)
